# Lese web_logs som Canvas_history
Først importerer eg alle nødvendige funksjonar:

In [162]:
import akvut
import pandas as pd


Så les eg `web_logs` frå det siste døgeret (det tok 3'49.8''):

In [3]:

logger = akvut.lag_logger('timer.log')
data = akvut.les_web_logs(logger)
print(data.info())


2025-02-11 08:11:00,778 - my_logger - INFO - Henta access_token OK.
2025-02-11 08:11:00,778 - my_logger - INFO - Henta access_token OK.
2025-02-11 08:11:00,788 - my_logger - INFO - payload er {"format": "csv", "since": "2025-02-10T08:11:00.788740Z"}
2025-02-11 08:11:00,788 - my_logger - INFO - payload er {"format": "csv", "since": "2025-02-10T08:11:00.788740Z"}
2025-02-11 08:11:00,793 - my_logger - INFO - Sender (inkrementel) spørjing til https://api-gateway.instructure.com/dap/query/canvas_logs/table/web_logs/data
2025-02-11 08:11:00,793 - my_logger - INFO - Sender (inkrementel) spørjing til https://api-gateway.instructure.com/dap/query/canvas_logs/table/web_logs/data


Har ikkje henta frå Azure


2025-02-11 08:11:01,605 - my_logger - INFO - Sjekker status på jobb 37c15967-ed7f-44bb-a0e2-eef70c94f084
2025-02-11 08:11:01,605 - my_logger - INFO - Sjekker status på jobb 37c15967-ed7f-44bb-a0e2-eef70c94f084
2025-02-11 08:11:07,123 - my_logger - INFO - Sjekker status på jobb 37c15967-ed7f-44bb-a0e2-eef70c94f084
2025-02-11 08:11:07,123 - my_logger - INFO - Sjekker status på jobb 37c15967-ed7f-44bb-a0e2-eef70c94f084
2025-02-11 08:11:12,686 - my_logger - INFO - Sjekker status på jobb 37c15967-ed7f-44bb-a0e2-eef70c94f084
2025-02-11 08:11:12,686 - my_logger - INFO - Sjekker status på jobb 37c15967-ed7f-44bb-a0e2-eef70c94f084
2025-02-11 08:11:18,165 - my_logger - INFO - Sjekker status på jobb 37c15967-ed7f-44bb-a0e2-eef70c94f084
2025-02-11 08:11:18,165 - my_logger - INFO - Sjekker status på jobb 37c15967-ed7f-44bb-a0e2-eef70c94f084
2025-02-11 08:11:23,697 - my_logger - INFO - Sjekker status på jobb 37c15967-ed7f-44bb-a0e2-eef70c94f084
2025-02-11 08:11:23,697 - my_logger - INFO - Sjekker st

Feil ved opplasting av sist oppdatert: ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect); [IM002] [Microsoft][ODBC Driver Manager] Invalid connection string attribute (0)')
<class 'pandas.core.frame.DataFrame'>
Index: 3012150 entries, 0 to 1377796
Data columns (total 25 columns):
 #   Column                              Dtype 
---  ------                              ----- 
 0   key.id                              object
 1   value.timestamp                     object
 2   value.user_id                       object
 3   value.real_user_id                  object
 4   value.course_id                     object
 5   value.quiz_id                       object
 6   value.discussion_id                 object
 7   value.conversation_id               object
 8   value.assignment_id                 object
 9   value.url                           object
 10  value.http_method                   object
 11  value.

Når eg les `web_logs` lagrer eg rådata i ei fil (`ny_history_alle.csv`). Men eg kan no sende desse vidare til nye rutiner der eg henter ut andre typer data.

In [ ]:

akvut.ny_history(data, logger)

## Alias
Eg prøver å lage statistikk over kven som opptrer som kven. Her har eg `user_id` på dei mest vanlege superbrukarane, og så kan eg lage ei liste over 

- kven dei har opptrådd som
- kor mange ganger dei har opptredd

NB! Det er mange av desse som er "Studentvising", slik at det er mange "user_id" som ikkje finst.


In [163]:
alias = data[~data['value.real_user_id'].isnull()]
adm_namn = {'2916': 'Aasmund', '12353': 'Johan', '3861': 'Casper', '3990': 'Dag Ove', '26737': 'Hege', '97393': 'Henrik', '35965': 'Jan Erik', '14186': 'Andreas Senneset Hove', '6': 'Ivar', '1703': 'Terje'}
adm_liste = []
for adm in ["1703", "2916", "12353", "97393", "3861", "14186", "35965", "98283", "5036", "6", "86785", "26737", "3990"]:
    adm_liste.append({'user': adm, 'count': alias[alias['value.real_user_id'] == adm]['value.user_id'].value_counts()})
for a in admliste:
    if len(a['count']) > 0:
        print(f"Administrator: {a['user']} ({adm_namn[a['user']]})")
        for index, item in a['count'].items():
            print(f"{index}: {item}")


Administrator: 2916 (Aasmund)
6799: 480
28799: 32
31653: 22
89604: 6
Administrator: 12353 (Johan)
35382: 38
22082: 29
Administrator: 26737 (Hege)
110437: 18
Administrator: 3990 (Dag Ove)
109344: 59
104760: 55
101467: 45


## Data (til history)
Så til det eg eigentleg skulle gjere: hente ut data til U4-prosjektet. Det er tre krav:
1. Feltet `user_id` må innehalde noko
2. Feltet `value.http_status` skal være `200`
3. Feltet `value.url` skal ikkje starte på "/api/"

In [164]:
utval = data[(data["value.user_id"].notnull()) & (data["value.http_status"].str.startswith("2")) & (~data["value.url"].str.startswith("/api/"))]


Det er litt usikkert kva vi skal bruke det til, men eg kan sende data til Håvard og be han plukke ut. 

In [113]:
utval.to_csv("history_250210_alle.csv")

Eg kan kanskje sende berre dei som er interessante (håper eg treff på utvalet):

In [165]:
redusert_utval = utval[['value.timestamp', 'value.user_id', 'value.course_id', 'value.url', 'value.web_application_controller', 'value.web_application_action', 'value.web_application_context_type']]


In [166]:
redusert_utval.to_csv("history_250210_redusert_utval.csv")

In [167]:
redusert_utval['dato'] = redusert_utval['value.timestamp'].apply(lambda x: x[0:10])

C:\Users\AKV\AppData\Local\Temp\ipykernel_2900\2458858629.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  redusert_utval['dato'] = redusert_utval['value.timestamp'].apply(lambda x: x[0:10])


Eg kan lage ei liste over kor mange aktive hendingar det er per student:

In [168]:
utval['value.user_id'].value_counts()

value.user_id
102235    848
94321     472
66147     432
103560    427
93294     390
         ... 
103655      1
4802        1
89604       1
93552       1
7930        1
Name: count, Length: 12256, dtype: int64

In [169]:
redusert_utval

,value.timestamp,value.user_id,value.course_id,value.url,value.web_application_controller,value.web_application_action,value.web_application_context_type,dato
18,2025-02-10T10:00:06.239Z,81990,NaN,/,users,user_dashboard,User,2025-02-10
19,2025-02-10T10:00:06.328Z,80600,23255,/courses/23255/files/3181872/preview,files,show,Course,2025-02-10
20,2025-02-10T10:00:07.037Z,50234,NaN,/page_views/8cc44091-1bf2-466b-9011-ec6e1486e6...,page_views,update,NaN,2025-02-10
25,2025-02-10T10:00:07.778Z,102555,29256,/courses/29256/modules/111346/collapse,context_modules,toggle_collapse,Course,2025-02-10
51,2025-02-10T10:00:12.817Z,32621,NaN,/groups/160998,groups,show,Group,2025-02-10
...,...,...,...,...,...,...,...,...
1377733,2025-02-11T05:04:03.891Z,33588,NaN,/login/session_token?return_to=https://hvl.ins...,login,session_token,NaN,2025-02-11
1377734,2025-02-11T05:04:03.906Z,33588,NaN,/,users,user_dashboard,User,2025-02-11
1377742,2025-02-11T05:04:04.775Z,33588,23255,/login/session_token?return_to=https://hvl.ins...,login,session_token,NaN,2025-02-11
1377745,2025-02-11T05:04:04.891Z,33588,23255,/login/session_token?return_to=https://hvl.ins...,login,session_token,NaN,2025-02-11


## Leit etter duplikat
Det neste vert å leite etter duplikat. Det vil sei: rader som har lik verdi i fleire celler. Då må eg eigentleg lage ei ny dataramme; eg går gjennom den første og finn like rader, og så legg eg denne raden (og antalet) i ei ny ramme. Vi prøver med noko enkelt:

`user_id`, `course_id`, `web_application_controller`, `web_application_action` og `dato` er like.


In [183]:
temp_duplikat = redusert_utval[["value.user_id", "value.course_id", "value.web_application_controller", "value.web_application_action", "value.web_application_context_type", "dato"]].value_counts()
duplikat = temp_duplikat.to_frame('frekvens').reset_index()

Og så lagrer eg til fil før eg ser på detaljane:

In [185]:
duplikat.to_csv('duplikat_250210.csv')

Først er eg på dei fem mest brukte linjene:

In [188]:
duplikat.head(5)

,value.user_id,value.course_id,value.web_application_controller,value.web_application_action,value.web_application_context_type,dato,frekvens
0,102235,28890,quizzes/quiz_submissions,backup,Course,2025-02-10,597
1,102206,30823,quizzes/quiz_submissions,backup,Course,2025-02-10,320
2,50889,30823,quizzes/quiz_submissions,backup,Course,2025-02-10,192
3,76609,30771,gradebooks,change_gradebook_column_size,Course,2025-02-10,159
4,94554,28892,quizzes/quiz_submissions,backup,Course,2025-02-10,129


Her har altså student 102235 gjort same kommando 597 ganger! For å finne ut kva det er må eg nesten sjekke med rådata (av di dette er ein quiz har eg ikkje tatt med data om quizen i utvalet):

In [194]:
student_102235 = data[(data['value.user_id'] == "102235") & (data['value.quiz_id'] == "18705")]

In [197]:
student_102235.to_csv('student_102235.csv', index=False)

In [187]:
duplikat['frekvens'].value_counts()

frekvens
1      32121
2      11702
3       5075
4       2713
5       1595
       ...  
63         1
62         1
59         1
320        1
597        1
Name: count, Length: 93, dtype: int64